<a href="https://colab.research.google.com/github/daamayar/MD_martini/blob/main/Martini%2Bcg2all.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

This is a Jupyter notebook for running Molecular Dynamics (MD) simulations of protein systems using OpenMM engine and Martini force field. This notebook is based on the implementations described in the paper "***Making it rain: Cloud-based molecular simulations for everyone***" ([link here](https://doi.org/10.1021/acs.jcim.1c00998)).



# **Settings for running this notebook**

Use the martini.dockerfile to build a Docker container with all the necessary dependencies for running this notebook.

## **Steps to Do It:**

### 1. Build the Docker Container
```bash
   docker build -t martini_img:latest -f martini.dockerfile .
```

### 2. Run the Container
```bash
   docker run --restart always --gpus all --name martini_inst -d -it -v $(pwd):/workspace -p 8888:8888 martini_img
```

### 3. Access the Notebook
- Once the container is running, copy the URL with the token from the terminal
- Open it in your web browser
- The notebook will be available in the `/workspace` directory

### 4. Run the Notebook
Two option are available : 
- Run the Notebook from the graphical interface in the web browser using the 'base' kernel
- Run the Notebook from the command line :
```
source activate base

nohup jupyter nbconvert --to notebook --execute Martini+cg2all.ipynb --output Martini+cg2all.ipynb > log_notebook &
```


## **Important Notes**

- Make sure Docker is installed on your system
- The `martini.dockerfile` contains all required dependencies
- Your current directory will be mounted as `/workspace` inside the container
- The notebook server will be accessible on port 8888

# **Load dependencies**

In [1]:
import sys
import tarfile
import os
import subprocess

clone_cmd = "git clone --depth 1 https://github.com/pablo-arantes/Making-it-rain/ temp"
os.system(clone_cmd)
os.system("cp -r temp/martini .")
os.system("rm -rf temp")

import sys
from biopandas.pdb import PandasPdb
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import os
import urllib.request
import numpy as np
import MDAnalysis as mda
import py3Dmol
import pytraj as pt
import platform
import scipy.cluster.hierarchy
from scipy.spatial.distance import squareform
import scipy.stats as stats
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import seaborn as sb
from statistics import mean, stdev
from pytraj import matrix
from matplotlib import colors
from IPython.display import set_matplotlib_formats
import subprocess
import warnings
warnings.filterwarnings('ignore')

!source activate cg2all

Cloning into 'temp'...
Updating files: 100% (652/652), done.


# **Loading the necessary input files**

Below, you should provide the name of the input file and the pathway of the folder containing it.

**Please, don't use spaces in the files and folders names, i.e. /app/MD_martini and so on.**

**Please, remove any water and replace the residue name HSD by HIS.**

In [2]:
filename = 'AKP6_WLIDMESLV_wt_frame_0.pdb'
Path = '/app/MD_martini/AKP6_WT'

In [5]:
workDir = Path

if os.path.exists(os.path.join(workDir)):
    pass
else:
    os.system("mkdir " + str(workDir))

foldername = 'martini'
top_folder = os.path.join(workDir, str(foldername))
top_folder_check = os.path.exists(top_folder)
if top_folder_check == False:
    os.system("cp -r ./martini " + str(workDir))
else:
    pass

outfnm = os.path.join(workDir, filename)

import numpy as np
import mdtraj as md
from six import PY2
from mdtraj.utils import ensure_type
from mdtraj.geometry.hbond import _prep_kabsch_sander_arrays
from mdtraj.geometry import _geometry
if PY2:
    from string import maketrans
else:
    maketrans = str.maketrans

SIMPLIFIED_CODE_TRANSLATION = maketrans('HGIEBTS ', 'HHHEECCC')
__all__ = ['compute_dssp']

def compute_dssp(traj, simplified=True):
    """Compute Dictionary of protein secondary structure (DSSP) secondary structure assignments
    Parameters
    ----------
    traj : md.Trajectory
        A trajectory
    simplified : bool, default=True
        Use the simplified 3-category assignment scheme. Otherwise the original
        8-category scheme is used.
    Returns
    -------
    assignments : np.ndarray, shape=(n_frames, n_residues), dtype=S1
        The assignments is a 2D array of character codes (see below), giving
        the secondary structure of each residue in each frame.
    Notes
    -----
    The DSSP assignment codes are:
       - 'H' : Alpha helix
       - 'B' : Residue in isolated beta-bridge
       - 'E' : Extended strand, participates in beta ladder
       - 'G' : 3-helix (3/10 helix)
       - 'I' : 5 helix (pi helix)
       - 'T' : hydrogen bonded turn
       - 'S' : bend
       - ' ' : Loops and irregular elements
    The simplified DSSP codes are:
       - 'H' : Helix. Either of the 'H', 'G', or 'I' codes.
       - 'E' : Strand. Either of the 'E', or 'B' codes.
       - 'C' : Coil. Either of the 'T', 'S' or ' ' codes.
    A special 'NA' code will be assigned to each 'residue' in the topology which
    isn't actually a protein residue (does not contain atoms with the names
    'CA', 'N', 'C', 'O'), such as water molecules that are listed as 'residue's
    in the topology.
    The implementation is based on DSSP-2.2.0, written by Maarten L. Hekkelman
    and distributed under the Boost Software license.
    References
    ----------
    .. [1] Kabsch W, Sander C (1983). "Dictionary of protein secondary
       structure: pattern recognition of hydrogen-bonded and geometrical
       features". Biopolymers 22 (12): 2577-637. doi:10.1002/bip.360221211
    """
    if traj.topology is None:
        raise ValueError('kabsch_sander requires topology')

    xyz, nco_indices, ca_indices, proline_indices, protein_indices \
        = _prep_kabsch_sander_arrays(traj)
    chain_ids = np.array([r.chain.index for r in traj.top.residues], dtype=np.int32)

    value = _geometry._dssp(xyz, nco_indices, ca_indices, proline_indices, chain_ids)
    if simplified:
        value = value.translate(SIMPLIFIED_CODE_TRANSLATION)

    n_frames = xyz.shape[0]
    n_residues = nco_indices.shape[0]
    if PY2:
        array = np.fromiter(value, dtype=np.dtype('S2'))
    else:
        array = np.fromiter(value, dtype=np.dtype('U2'))

    array = array.reshape(n_frames, n_residues)
    array[:, np.logical_not(protein_indices)] = 'NA'
    return array

starting = os.path.join(workDir, "starting.pdb")
starting2 = os.path.join(workDir, "starting2.pdb")
starting_end = os.path.join(workDir, "starting_end.pdb")
ppdb = PandasPdb().read_pdb(outfnm)
ppdb.df['ATOM'] = ppdb.df['ATOM']
ppdb.df['HETATM'] = ppdb.df['HETATM'][ppdb.df['HETATM']['residue_name'] == 'HOH']
ppdb.df['ATOM'] = ppdb.df['ATOM'][ppdb.df['ATOM']['atom_name'] != 'OXT']
ppdb.df['ATOM']= ppdb.df['ATOM'][ppdb.df['ATOM']['element_symbol'] != 'H']
ppdb.to_pdb(path=starting, records=['ATOM', 'HETATM'], gz=False, append_newline=True)

from Bio.PDB import is_aa
from Bio.PDB import PDBParser, PDBIO, Select

class ProtSelect(Select):
    def accept_residue(self, residue):
        print(f"{residue} -> {is_aa(residue)}")
        return is_aa(residue, standard=True)

from Bio import PDB

pdb_ini = PDBParser().get_structure("pdb", starting)
io = PDBIO()
io.set_structure(pdb_ini)
io.save(starting2, ProtSelect());

pdb4amber_cmd = "pdb4amber -i " + str(starting2) + " -o " + str(starting_end) + " -p"
original_stdout = sys.stdout # Save a reference to the original standard output

with open('pdb4amber.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(pdb4amber_cmd)
    sys.stdout = original_stdout # Reset the standard output to its original value

subprocess.run(["chmod 700 pdb4amber.sh"], shell=True)
subprocess.run(["./pdb4amber.sh"], shell=True,)


pdb_ss = md.load_pdb(starting2)
ss = compute_dssp(pdb_ss, simplified=True,)

my_string = ''.join([str(item) for sublist in ss for item in sublist])

<Residue GLY het=  resseq=1 icode= > -> True
<Residue SER het=  resseq=2 icode= > -> True
<Residue HIS het=  resseq=3 icode= > -> True
<Residue SER het=  resseq=4 icode= > -> True
<Residue MET het=  resseq=5 icode= > -> True
<Residue ARG het=  resseq=6 icode= > -> True
<Residue TYR het=  resseq=7 icode= > -> True
<Residue PHE het=  resseq=8 icode= > -> True
<Residue PHE het=  resseq=9 icode= > -> True
<Residue THR het=  resseq=10 icode= > -> True
<Residue SER het=  resseq=11 icode= > -> True
<Residue VAL het=  resseq=12 icode= > -> True
<Residue SER het=  resseq=13 icode= > -> True
<Residue ARG het=  resseq=14 icode= > -> True
<Residue PRO het=  resseq=15 icode= > -> True
<Residue GLY het=  resseq=16 icode= > -> True
<Residue ARG het=  resseq=17 icode= > -> True
<Residue GLY het=  resseq=18 icode= > -> True
<Residue GLU het=  resseq=19 icode= > -> True
<Residue PRO het=  resseq=20 icode= > -> True
<Residue ARG het=  resseq=21 icode= > -> True
<Residue PHE het=  resseq=22 icode= > -> Tr


Summary of pdb4amber for: /app/MD_martini/AKP6_WT/starting2.pdb

----------Chains
The following (original) chains have been found:
A
B
C

---------- Alternate Locations (Original Residues!))

The following residues had alternate locations:
None
-----------Non-standard-resnames


---------- Missing heavy atom(s)

None


# **Showing 3D structure**


## **Visualization parameters**

In [4]:
color = "rainbow" # "gray", "rainbow"
show_sidechains = False # boolean
show_mainchains = False # boolean
show_box = True # boolean
opacity = 0.3 # min:0, max:1

In [5]:
import warnings
warnings.filterwarnings('ignore')
import py3Dmol

def show_pdb(show_sidechains=False, show_mainchains=False, show_box = False, color="rainbow"):
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
    view.addModel(open(starting2,'r').read(),'pdb')

    if color == "gray":
        view.setStyle({'cartoon':{}})
    elif color == "rainbow":
        view.setStyle({'cartoon': {'color':'spectrum'}})

    if show_sidechains:
        BB = ['C','O','N']
        view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                            {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"PW"}]},
                            {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
        view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                            {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    if show_mainchains:
        BB = ['C','O','N','CA']
        view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})

    if show_box:
        view.addSurface(py3Dmol.SAS, {'opacity': opacity, 'color':'white'})
    view.zoomTo()
    return view

print("Secondary Structure = " + str(my_string))
show_pdb(show_sidechains, show_mainchains, show_box, color).show()

Secondary Structure = CEEEEEEEEEEECCCCCCCCEEEEEEEECCEEEEEEECCCCCCCCEECCHHHHCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCEEEEEEEEEEECCCCEEEEEEEEEECCEEEEEECCCCCCEEECCHHHHHHHHHHHHHCHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCEEEEEEEEECCCEEEEEEEEEEECCCCCEEEEEECCEECCCCEEECCCEECCCCCEEEEEEEEECCCCCCCCEEEEECCCCCCCEEECCCCCCECCEEEEEECCCCCCCCCEEEEEEEEEEECCCCEEEEEECCEECCCCEECCCEEECCCEEEEEEEEEECCCCCCCEEEEEECCCCCCCEEEEECCCCCCCCCCCCC


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# **Parameters to generate the Martini topology**

In [6]:
import glob

top_martini = os.path.join(workDir, "martini.top")
top_temp = os.path.join(workDir, "martini_temp.top")
gro_martini = os.path.join(workDir, "system.gro")
pdb_nw_martini = os.path.join(workDir, "martini.pdb")
pdb_martini = os.path.join(workDir, "ions.pdb")

force_field = "martini3001" # martini3001, martini22
# Position Restraints:
position_restraints = "backbone" # none, all, backbone
# Position restraints force constant in kJ/mol (default: 1000):
posres_force = 1000 # min:0, max:2000
secondary_structure_handling = "yes" # yes, no
if secondary_structure_handling == "yes":
    ss_h = my_string
else:
    ss_h = ''

# Protein elastic network:
write_elastic_bonds = "False" # True, False
# Elastic bond force constant Fc in kJ/mol (default: 500):
elastic_force = 500 # min:0, max:2000
# Elastic bond lower cutoff (default: 0):
elastic_lower = 0 # min:0, max:1 
# Elastic bond upper cutoff (default: 0.9):
elastic_upper = 0.9 # min:0, max:1
if write_elastic_bonds == "True":
    elastic = " -elastic -ef " + str(elastic_force) + " -el " + str(elastic_lower) + " -eu " + str(elastic_upper)
else:
    elastic = ''

side_chain_corrections = "yes" # yes, no
if side_chain_corrections == "yes":
    scfix = " -scfix"
else:
    scfix = ''
cystein_bonds = "yes" # yes, no
if cystein_bonds == "yes":
    cys = "auto"
else:
    cys = "none"

if os.path.exists(top_martini):
    os.remove(top_martini)
    os.remove(gro_martini)
    os.remove(pdb_nw_martini)
else:
    pass

folder_workdir = "cd " + str(workDir)
martinize2 = "martinize2 -f " + str(starting2) + " -x " + str(pdb_nw_martini)  + " -ss " + str(ss_h) + " -ff " + str(force_field) +  str(elastic) + " -p " + str(position_restraints) +  " -pf " + str(posres_force) + " -cys " + str(cys) + str(scfix) +  " -o " + str(top_martini) + " -maxwarn 10 -ignh"

original_stdout = sys.stdout # Save a reference to the original standard output
with open('martinize.sh', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(folder_workdir)
    print(martinize2)
    sys.stdout = original_stdout # Reset the standard output to its original value

os.system("chmod 700 martinize.sh")
os.system("bash martinize.sh")

    INFO - general - Read 3 molecules from PDB file /app/MD_martini/AKP6_WT/starting2.pdb
    INFO - step - Guessing the bonds.
    INFO - general - 3 molecules after guessing bonds
    INFO - step - Repairing the graph.
    INFO - general - Applying modification N-ter to residue A-GLY1
    INFO - general - Applying modification C-ter to residue A-PRO276
    INFO - general - Applying modification N-ter to residue B-ILE277
    INFO - general - Applying modification C-ter to residue B-MET375
    INFO - general - Applying modification N-ter to residue C-TRP376
    INFO - general - Applying modification C-ter to residue C-VAL384
    INFO - step - Dealing with modifications.
    INFO - general - Identified the modifications ['N-ter'] on residues ['GLY1', 'GLY1', 'GLY1', 'GLY1']
    INFO - general - Identified the modifications ['C-ter'] on residues ['PRO276', 'PRO276', 'PRO276']
    INFO - general - Identified the modifications ['N-ter'] on residues ['ILE277', 'ILE277', 'ILE277', 'ILE277']


0

# **Parameters to solvate the system**

In [7]:
box_type = "cubic" # square, cubic, optimal

# Minimum distance from the protein to box edge (nm):
size_box = 5 # min:0, max:15 

# ATTENTION : Give the concentration in Molar units (NaCl):
Concentration = 0.15 # min:0, max:2

In [8]:
os.system("insane -o " + str(gro_martini) + " -p " + str(top_temp) + " -f "  + str(pdb_nw_martini) + " -d " + str(size_box) + " -sol W -salt " + str(Concentration) + " -charge auto -pbc " + str(box_type))

with open(top_martini, 'r') as f:
    lines = f.readlines()
# Remove the first line
lines.pop(0)
lines.pop(0)
if force_field == "martini3001":
    new_lines = ['#include "martini/martini_v3.0.0.itp"\n', '#include "martini/martini_v3.0.0_ions.itp"\n', '#include "martini/martini_v3.0.0_solvents.itp"\n']
elif force_field == "martini22":
    new_lines = ['#include "martini/martini_v2.2.itp"\n', '#include "martini/martini_v2.2_ions.itp"\n', '#include "martini/martini_v2.2_aminoacids.itp"\n']
else:
    pass
lines = new_lines + lines
# Write the updated content back to the file
with open(top_martini, 'w') as f:
    f.writelines(lines)


if Concentration == 0:
    with open(top_temp, "r") as file:
        lines = file.readlines()
    last_line = lines[-1].split()
    if last_line[0] == "W":
        with open(top_temp) as f_in, open(top_martini, "a") as f_out :
            for row in f_in.readlines()[-1:] :
                f_out.write(row)
    else:
        with open(top_temp) as f_in, open(top_martini, "a") as f_out :
            for row in f_in.readlines()[-2:] :
                f_out.write(row)
else:
    with open(top_temp) as f_in, open(top_martini, "a") as f_out :
        for row in f_in.readlines()[-3:] :
            f_out.write(row)

universe = mda.Universe(gro_martini)
with mda.Writer(pdb_martini) as pdb:
    pdb.write(universe)

os.system("rm " + str(workDir) + "/*#")
os.remove(top_temp)

gro_check = os.path.exists(gro_martini)
top_check = os.path.exists(top_martini)

if gro_check == True and top_check == True:
    print("Successfully generated topology! :-)")
else:
    print("ERROR: Check your input file! ")

; NDX Solute 1 963
; Charge of protein: -11.000000
; NDX Membrane 964 0
; Charge of membrane: 0.000000
; Total charge: -11.000000
; NDX Solvent 964 19644
; NDX System 1 19644
; "I mean, the good stuff is just INSANE" --Julia Ormond


Successfully generated topology! :-)


## **Let's take a look on our simulation box**

In [9]:
import warnings
warnings.filterwarnings('ignore')
import py3Dmol
show_waters = False 
opacity = 0.4 # min:0, max:1

if show_waters == False:
    input_file = pdb_nw_martini
else:
    input_file = pdb_martini

def show_pdb(color="rainbow"):
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
    view.addModel(open(input_file,'r').read(),'pdb')
    view.setStyle({'sphere': {'color':'spectrum'}})
    view.addSurface(py3Dmol.SAS, {'opacity': opacity, 'color':'white'})
    view.zoomTo()
    return view

show_pdb(color).show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.


# **Equilibrating the simulation box**

MD equilibration protocol can be designed to equilibrate both temperature and pressure throughout the simulation box while preserving the protein experimental conformation. In addition, we also allow the solvent to accomodate around the protein, creating proper solvation layers.

Below, we will set up the MD equilibration parameters, such as temperature, pressure and the desired simulation time. We will define the force constant used to restraint protein heavy-atoms in place and the frequency at which we want to save atomic coordinates in a trajectory file (.dcd).

After you are done, you can run the next 2 cells to equilibrate your system on NPT (first) and/or NVT ensembles.

## **Parameters for MD Equilibration (NPT and/or NVT):**

In [6]:
# remove whitespaces
Jobname = 'AKP6_WT'
Minimization_steps = "5000" # 1000, 5000, 10000, 20000, 50000, 100000

# Integration time (in femtoseconds):
Integration_timestep = "5" # 1, 5, 10, 20, 30, 40
dt_eq = Integration_timestep

# Simulation time (in nanoseconds) for NPT equilibration:
Time = "5"
stride_time_eq_npt = Time

NVT_equi = False

# Simulation time (in nanoseconds) for NVT equilibration:
Time = "5" 
stride_time_eq_nvt = Time

# Temperature (in Kelvin) and Pressure (in bar)
Temperature = "310"
temperature_eq = Temperature
Pressure = 1
pressure_eq = Pressure

# Position restraints force constant (in kJ/mol):
# Force_constant = 500 # min:0, max:2000

# Frequency to write the trajectories files (in picoseconds):

Write_the_trajectory = "100" # 10, 100, 200, 500, 1000
write_the_trajectory_eq = Write_the_trajectory

# Frequency to write the log file (in picoseconds):

Write_the_log = "100" # 10, 100, 200, 500, 1000
write_the_log_eq = Write_the_log

## **Runs an Equilibration coarse-grained MD simulation (NPT and/or NVT ensembles)**

In [7]:
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import martini_openmm as martini
from mdtraj.reporters import XTCReporter
import pytraj as pt
import parmed
from sys import stdout, exit, stderr
import os, math, fnmatch

# Defining MD simulation parameters

jobname = os.path.join(workDir, Jobname)
coordinatefile = os.path.join(workDir, "system.gro")
pdbfile = os.path.join(workDir, "ions.pdb")
topologyfile = os.path.join(workDir, "martini.top")

time_ps = float(stride_time_eq_npt)*1000
simulation_time = float(time_ps)*picosecond		# in ps
dt = int(dt_eq)*femtosecond
temperature = float(temperature_eq)*kelvin
savcrd_freq = int(write_the_trajectory_eq)*picosecond
print_freq  = int(write_the_log_eq)*picosecond

pressure	= float(pressure_eq)*bar

# restraint_fc = int(Force_constant) # kJ/mol

nsteps  = int(simulation_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))

#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file

# def restraints(system, crd, fc, restraint_array):

# 	boxlx = system.getDefaultPeriodicBoxVectors()[0][0].value_in_unit(nanometers)
# 	boxly = system.getDefaultPeriodicBoxVectors()[1][1].value_in_unit(nanometers)
# 	boxlz = system.getDefaultPeriodicBoxVectors()[2][2].value_in_unit(nanometers)

# 	if fc > 0:
# 		# positional restraints for all heavy-atoms
# 		posresPROT = CustomExternalForce('k*periodicdistance(x, y, z, x0, y0, z0)^2;')
# 		posresPROT.addPerParticleParameter('k')
# 		posresPROT.addPerParticleParameter('x0')
# 		posresPROT.addPerParticleParameter('y0')
# 		posresPROT.addPerParticleParameter('z0')

# 		for atom1 in restraint_array:
# 			atom1 = int(atom1)

# 			xpos  = crd.positions[atom1].value_in_unit(nanometers)[0]
# 			ypos  = crd.positions[atom1].value_in_unit(nanometers)[1]
# 			zpos  = crd.positions[atom1].value_in_unit(nanometers)[2]

# 			posresPROT.addParticle(atom1, [fc, xpos, ypos, zpos])

# 		system.addForce(posresPROT)

# 	return system
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tCoordinate file = " + str(coordinatefile))
print("\tPDB file = " + str(pdbfile))
print("\tTopology file = " + str(topologyfile))

print("\n\tSimulation_time = " + str(simulation_time))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps))

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
#############################################

# get any defines
defines = {}
try:
	with open("defines.txt") as def_file:
		for line in def_file:
			line = line.strip()
			defines[line] = True
except FileNotFoundError:
	pass

print("\n> Setting the system:\n")

print("\t- Reading topology and structure file...")
gro = GromacsGroFile(coordinatefile)
top = martini.MartiniTopFile(topologyfile, periodicBoxVectors=gro.getPeriodicBoxVectors(), defines=defines)

print("\t- Creating system and setting parameters...")
friction = 10.0 / picosecond
system = top.create_system(nonbonded_cutoff=1.1*nanometer)

# print("\t- Applying restraints. Force Constant = " + str(Force_constant) + "kJ/mol")
# pt_system = pt.iterload(coordinatefile, pdb)
# pt_topology = pt_system.top
# restraint_array = pt.select_atoms('!(:SOL) & !(:NA) & !(:CL) & !(:MG) & !(:K)', pt_topology)

# system = restraints(system, gro, restraint_fc, restraint_array)

# print("\t- Setting barostat...")
# system.addForce(MonteCarloBarostat(pressure, temperature))

print("\t- Setting integrator...")
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setRandomNumberSeed(0)
# integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(top.topology, system, integrator)
simulation.context.setPositions(gro.positions)

parmed_structure = parmed.openmm.load_topology(top.topology, system, gro.positions)

parmed_structure.save(os.path.join(workDir, 'system_openMM.pdb'), overwrite=True)
with open(os.path.join(workDir, 'system_openMM.xml'), 'w') as output:
  	output.write(XmlSerializer.serialize(system))

print("\t- Energy minimization: " + str(Minimization_steps) + " steps")
simulation.minimizeEnergy(tolerance=1.0, maxIterations=int(Minimization_steps))

print("\t-> Potential Energy = " + str(simulation.context.getState(getEnergy=True).getPotentialEnergy()))

print("\t- Setting initial velocities...")
simulation.context.setVelocitiesToTemperature(temperature)

#############################################
# Running Equilibration on NPT ensemble

time_ps = float(stride_time_eq_npt)*1000
simulation_time = float(time_ps)*picosecond		# in ps
dt = int(dt_eq)*femtosecond
temperature = float(temperature_eq)*kelvin
savcrd_freq = int(write_the_trajectory_eq)*picosecond
print_freq  = int(write_the_log_eq)*picosecond
pressure	= float(pressure_eq)*bar

nsteps  = int(simulation_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))

print("\tPressure = " + str(pressure))
print("\t- Setting barostat...")
system.addForce(MonteCarloBarostat(pressure, temperature))
'''
print("\n> Loading previous state from equilibration > " + rst_file_nvt + " <")
with open(rst_file_npt, 'r') as f:
	simulation.context.setState(XmlSerializer.deserialize(f.read()))
	currstep = int((1-1)*nsteps)
	currtime = currstep*dt.in_units_of(picosecond)
	simulation.currentStep = currstep
	simulation.context.setTime(currtime)
	print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")
'''
dcd_file = jobname + "_eq_npt.dcd"
log_file = jobname + "_eq_npt.log"
rst_file_npt = jobname + "_eq_npt.rst"
prv_rst_file = jobname + "_eq_npt.rst"
pdb_file = jobname + "_eq_npt.pdb"

# Creating a trajectory file and reporters
dcd = DCDReporter(dcd_file, nsavcrd)
firstdcdstep = (nsteps) + nsavcrd
dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # charmm doesn't like first step to be 0

simulation.reporters.append(dcd)
simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=nsteps, remainingTime=True, separator='\t\t'))
simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

print("\n> Simulating " + str(nsteps) + " steps...")
simulation.step(nsteps)

simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.

##################################
# Writing last frame information of stride
print("\n> Writing state file (" + str(rst_file_npt) + ")...")
state = simulation.context.getState( getPositions=True, getVelocities=True )
with open(rst_file_npt, 'w') as f:
	f.write(XmlSerializer.serialize(state))

last_frame = int(nsteps/nsavcrd)
print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")

#############################################

if NVT_equi:
	# Running Equilibration on NVT ensemble

	dcd_file = jobname + "_eq_nvt.dcd"
	log_file = jobname + "_eq_nvt.log"
	rst_file_nvt = jobname + "_eq_nvt.rst"
	prv_rst_file = jobname + "_eq_nvt.rst"
	pdb_file = jobname + "_eq_nvt.pdb"

	# Creating a trajectory file and reporters
	dcd = DCDReporter(dcd_file, nsavcrd)
	firstdcdstep = (nsteps) + nsavcrd
	dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # charmm doesn't like first step to be 0

	simulation.reporters.append(dcd)
	simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=nsteps, remainingTime=True, separator='\t\t'))
	simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

	print("\n> Simulating " + str(nsteps) + " steps...")
	simulation.step(nsteps)

	simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.

	##################################
	# Writing last frame information of stride
	print("\n> Writing state file (" + str(rst_file_nvt) + ")...")
	state = simulation.context.getState( getPositions=True, getVelocities=True )
	with open(rst_file_nvt, 'w') as f:
		f.write(XmlSerializer.serialize(state))

	last_frame = int(nsteps/nsavcrd)
	print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
	positions = simulation.context.getState(getPositions=True).getPositions()
	PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))


> Simulation details:

	Job name = /app/MD_martini/AKP6_WT/AKP6_WT
	Coordinate file = /app/MD_martini/AKP6_WT/system.gro
	PDB file = /app/MD_martini/AKP6_WT/ions.pdb
	Topology file = /app/MD_martini/AKP6_WT/martini.top

	Simulation_time = 5000.0 ps
	Integration timestep = 5 fs
	Total number of steps = 1000000

	Save coordinates each 100 ps
	Print in log file each 100 ps

	Temperature = 310.0 K

> Setting the system:

	- Reading topology and structure file...
	- Creating system and setting parameters...
	- Setting integrator...
	- Energy minimization: 5000 steps
	-> Potential Energy = -587670.6259765625 kJ/mol
	- Setting initial velocities...
	Pressure = 1.0 bar
	- Setting barostat...

> Simulating 1000000 steps...
#"Progress (%)"		"Step"		"Speed (ns/day)"		"Time Remaining"
2.0%		20000		0		--
4.0%		40000		913		7:34
6.0%		60000		920		7:21
8.0%		80000		930		7:07
10.0%		100000		930		6:57
12.0%		120000		937		6:45
14.0%		140000		937		6:36
16.0%		160000		934		6:28
18.0%		180000		932		6:20
20


# **Running a Production MD simulation**

Finally, we will proceed with the Production simulation itself using the equilibrated system coordinates as input structure.

Note that we will use here a *.rst state file* , which contains atomic velocities and positions from the last frame of the equilibration simulation, guaranteeing that our production simulation begins from a thermodynamically equilibrated system.

Another important information here is the **Number_of_strides** and the **Stride_Time**. In this notebook, we simulate a defined number of *strides*, so the **simulation time = Number_of_strides*Stride_Time**. For example, we can simulate 100ns by setting *Number_of_strides=10* and *Stride_Time=10 ns*.

**Important: at the end of the Production simulation, we concatenate all strides to create a complete trajectory file which can be visualized and analyzed**

The idea behind this approach is to make use of intermitent computation periods in cloud-based computing solutions (notably Google Colab).

## **Parameters for MD Production protocol:**
**Important**: 

The **Jobname** should be the same you have been used to run your simulation in the previous step.

If you are going to continue your simulation, all the parameters should be the same, including the **Jobname**. The only parameter you should change is the **Number of strides**, which you will increase.



In [5]:
# Equilibrated_PDB = '1aki_equil.pdb'
# State_file = '1aki_equil.rst'

workDir = Path
restart = True
if restart: Jobname = 'AKP6_WT'

# Simulation time (in nanoseconds), number of strides (integers) and integration timestep (in femtoseconds):
Stride_Time = "10"
stride_time_prod = Stride_Time
Number_of_strides = "30"
nstride = Number_of_strides
Integration_timestep = "5" # 5, 10, 20, 30, 40
dt_prod = Integration_timestep

# Temperature (in Kelvin) and Pressure (in bar)
Temperature = "310"
temperature_prod = Temperature
Pressure = 1 
pressure_prod = Pressure

# Frequency to write the trajectory file (in picoseconds):
Write_the_trajectory = "200" # 10, 100, 200, 500, 1000
write_the_trajectory_prod = Write_the_trajectory

# Frequency to write the log file (in picoseconds):
Write_the_log = "100" # 10, 100, 200, 500, 1000
write_the_log_prod = Write_the_log
continue_simulation = "yes" # yes, no

## **Runs a Production coarse-grained MD simulation (NPT ensemble) after equilibration**


In [6]:
import openmm as mm
from openmm import *
from openmm.app import *
from openmm.unit import *
import martini_openmm as martini
from mdtraj.reporters import XTCReporter
import pytraj as pt

from sys import stdout, exit, stderr
import os, math, fnmatch
#############################################

jobname = os.path.join(workDir, str(Jobname))
coordinatefile = os.path.join(workDir, "system.gro")
pdbfile = os.path.join(workDir, "ions.pdb")
topologyfile = os.path.join(workDir, "martini.top")

stride_time_ps = float(stride_time_prod)*1000
stride_time = float(stride_time_ps)*picosecond
nstride = int(Number_of_strides)
dt = int(dt_prod)*femtosecond
temperature = float(temperature_prod)*kelvin
savcrd_freq = int(write_the_trajectory_prod)*picosecond
print_freq  = int(write_the_log_prod)*picosecond

pressure	= float(pressure_prod)*bar

simulation_time = stride_time*nstride
nsteps  = int(stride_time.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nprint  = int(print_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
nsavcrd = int(savcrd_freq.value_in_unit(picosecond)/dt.value_in_unit(picosecond))
firststride = 1 # must be integer

# get any defines
defines = {}
try:
	with open("defines.txt") as def_file:
		for line in def_file:
			line = line.strip()
			defines[line] = True
except FileNotFoundError:
	pass

#############################################
# Defining functions to use below:
def backup_old_log(pattern, string):
	result = []
	for root, dirs, files in os.walk("./"):
		for name in files:
			if fnmatch.fnmatch(name, pattern):

				try:
					number = int(name[-2])
					avail = isinstance(number, int)
					#print(name,avail)
					if avail == True:
						result.append(number)
				except:
					pass

	if len(result) > 0:
		maxnumber = max(result)
	else:
		maxnumber = 0

	backup_file = "\#" + string + "." + str(maxnumber + 1) + "#"
	os.system("mv " + string + " " + backup_file)
	return backup_file
##############################################

#############################################
print("\n> Simulation details:\n")
print("\tJob name = " + jobname)
print("\tCoordinate file = " + str(coordinatefile))
print("\tPDB file = " + str(pdbfile))
print("\tTopology file = " + str(topologyfile))

print("\n\tSimulation_time = " + str(stride_time*nstride))
print("\tIntegration timestep = " + str(dt))
print("\tTotal number of steps = " +  str(nsteps*nstride))
print("\tNumber of strides = " + str(nstride) + " (" + str(stride_time) + " in each stride)")

print("\n\tSave coordinates each " + str(savcrd_freq))
print("\tSave checkpoint each " + str(savcrd_freq))
print("\tPrint in log file each " + str(print_freq))

print("\n\tTemperature = " + str(temperature))
print("\tPressure = " + str(pressure))
#############################################

print("\n> Setting the system:\n")

if continue_simulation == "yes":
	for i in range(1,int(nstride)+1):
		name = str(jobname) + "_" + str(i) + "_prod_npt.pdb"
		name2 = str(jobname) + "_" + str(i) + "_prod_npt.rst"
		if os.path.exists(name) and os.path.exists(name2):
			last_stride_pdb = name
			last_stride_rst = name2
		else:
			pass
	gro = PDBFile(last_stride_pdb)
	with open(os.path.join(workDir, 'system_openMM.xml')) as input:
  		system = XmlSerializer.deserialize(input.read())
	top = gro.getTopology()
else:
	top = top.topology

print("\t- Setting integrator...")
friction = 10.0 / picosecond
integrator = LangevinIntegrator(temperature, friction, dt)
integrator.setRandomNumberSeed(0)
simulation = Simulation(top, system, integrator)
simulation.context.setPositions(gro.positions)

print("\tPressure = " + str(pressure))
print("\t- Setting barostat...")
system.addForce(MonteCarloBarostat(pressure, temperature))

# Opening a loop of extension NSTRIDE to simulate the entire STRIDE_TIME*NSTRIDE
for n in range(1, nstride + 1):

	print("\n\n>>> Simulating Stride #" + str(n) + " <<<")
	rst_file_npt = jobname + "_eq_npt.rst"
	dcd_file = jobname + "_" + str(n) + "_prod_npt.dcd"
	log_file = jobname + "_" + str(n) + "_prod_npt.log"
	rst_file = jobname + "_" + str(n) + "_prod_npt.rst"
	prv_rst_file = jobname + "_" + str(n-1) + "_prod_npt.rst"
	pdb_file = jobname + "_" + str(n) + "_prod_npt.pdb"

	if os.path.exists(rst_file):
		print("> Stride #" + str(n) + " finished (" + rst_file + " present). Moving to next stride... <")
		continue

	if n == 1:
		print("\n> Loading previous state from equilibration > " + rst_file_npt + " <")
		with open(rst_file_npt, 'r') as f:
			simulation.context.setState(XmlSerializer.deserialize(f.read()))
			currstep = int((n-1)*nsteps)
			currtime = currstep*dt.in_units_of(picosecond)
			simulation.currentStep = currstep
			simulation.context.setTime(currtime)
			print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")

	else:
		print("> Loading previous state from > " + prv_rst_file + " <")
		with open(prv_rst_file, 'r') as f:
			simulation.context.setState(XmlSerializer.deserialize(f.read()))
			currstep = int((n-1)*nsteps)
			currtime = currstep*dt.in_units_of(picosecond)
			simulation.currentStep = currstep
			simulation.context.setTime(currtime)
			print("> Current time: " + str(currtime) + " (Step = " + str(currstep) + ")")


	dcd = DCDReporter(dcd_file, nsavcrd)
	firstdcdstep = (currstep) + nsavcrd
	dcd._dcd = DCDFile(dcd._out, simulation.topology, simulation.integrator.getStepSize(), firstdcdstep, nsavcrd) # first step should not be 0

	simulation.reporters.append(dcd)
	simulation.reporters.append(StateDataReporter(stdout, nprint, step=True, speed=True, progress=True, totalSteps=(nsteps*nstride), remainingTime=True, separator='\t\t'))
	simulation.reporters.append(StateDataReporter(log_file, nprint, step=True, kineticEnergy=True, potentialEnergy=True, totalEnergy=True, temperature=True, volume=True, speed=True))

	print("\n> Simulating " + str(nsteps) + " steps... (Stride #" + str(n) + ")")
	simulation.step(nsteps)

	simulation.reporters.clear() # remove all reporters so the next iteration don't trigger them.

	##################################
	# Writing last frame information of stride
	print("\n> Writing state file (" + str(rst_file) + ")...")
	state = simulation.context.getState( getPositions=True, getVelocities=True )
	with open(rst_file, 'w') as f:
		f.write(XmlSerializer.serialize(state))

	last_frame = int(nsteps/nsavcrd)
	print("> Writing coordinate file (" + str(pdb_file) + ", frame = " + str(last_frame) + ")...")
	positions = simulation.context.getState(getPositions=True).getPositions()
	PDBFile.writeFile(simulation.topology, positions, open(pdb_file, 'w'))

print("\n> Finished!\n")


> Simulation details:

	Job name = /app/MD_martini/AKP6_WT/AKP6_WT
	Coordinate file = /app/MD_martini/AKP6_WT/system.gro
	PDB file = /app/MD_martini/AKP6_WT/ions.pdb
	Topology file = /app/MD_martini/AKP6_WT/martini.top

	Simulation_time = 300000.0 ps
	Integration timestep = 5 fs
	Total number of steps = 60000000
	Number of strides = 30 (10000.0 ps in each stride)

	Save coordinates each 200 ps
	Save checkpoint each 200 ps
	Print in log file each 100 ps

	Temperature = 310.0 K
	Pressure = 1.0 bar

> Setting the system:

	- Setting integrator...
	Pressure = 1.0 bar
	- Setting barostat...


>>> Simulating Stride #1 <<<
> Stride #1 finished (/app/MD_martini/AKP6_WT/AKP6_WT_1_prod_npt.rst present). Moving to next stride... <


>>> Simulating Stride #2 <<<
> Stride #2 finished (/app/MD_martini/AKP6_WT/AKP6_WT_2_prod_npt.rst present). Moving to next stride... <


>>> Simulating Stride #3 <<<
> Stride #3 finished (/app/MD_martini/AKP6_WT/AKP6_WT_3_prod_npt.rst present). Moving to next stride.

# **Concatenate and align the trajectory**

**Important**: The **Path**, **Jobname**, **Number of strides**, **stride time** and **trajectory saved frequency** should be the same you have been used to run your simulation in the previous steps.

The code will save two trajectories: One with waters and other without waters.

## **Parameters for concatenation**

In [ ]:
Path = '/app/MD_martini/AKP6_WT'
workDir = Path
restart = False
if restart: Jobname = "AKP6_WT"
# force_field = "martini3001" # martini22p, martini3001, martini22
Skip = "1" # 1, 2, 5, 10, 20, 50
stride_traj = Skip
# Output_format = "dcd" # dcd, pdb, trr, xtc
first_stride = "1"
Number_of_strides = "100" 
nstride = int(Number_of_strides)
stride_time = "300" 
trajectory_saved_frequency = "200" # 10, 100, 200, 500, 1000
Write_trajectory = trajectory_saved_frequency
traj_save_freq = trajectory_saved_frequency
# Remove_waters = "no" # yes, no
# stride_id_as_ref_for_alignment = "1" 
# output_prefix = first_stride+"-"+str(int(first_stride)+nstride-1)

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms

Equilibrated_PDB = Jobname + "_eq_npt.pdb"
stride_time_ps = float(stride_time)*1000
simulation_time_analysis = stride_time_ps*nstride
simulation_ns = float(stride_time)*int(Number_of_strides)
number_frames = int(simulation_time_analysis)/int(traj_save_freq)
number_frames_analysis = number_frames/int(stride_traj)

nw_dcd = os.path.join(workDir, str(Jobname) + "_cg_nw.dcd")
nw_pdb = os.path.join(workDir, str(Jobname) +  "_nw.pdb")
backmap_dcd = os.path.join(workDir, str(Jobname) + "_100_frames_nw.dcd")
whole_pdb = os.path.join(workDir, str(Jobname) +  "_whole.pdb")
whole_dcd = os.path.join(workDir, str(Jobname) +  "_cg_whole.dcd")
template =  os.path.join(workDir, str(Jobname) + '_%s_prod_npt.dcd')
pdb = os.path.join(workDir, Equilibrated_PDB)

# if force_field == "martini3001":
#   mask_mda = "not (resname W or name NA+ or name CL-)"
# elif force_field == "martini22":
#   mask_mda = "not (resname W or name NA+ or name CL-)"
# else:
#   pass

mask_mda = "not (resname W or name NA+ or name CL-)"

flist = [template % str(i) for i in range(int(first_stride), int(first_stride) + nstride)]
ref = [template % int(1)]

u1 = mda.Universe(pdb, flist)
u2 = mda.Universe(pdb, ref)

num_frames = len(u1.trajectory)
if num_frames >= 10:
  stride_backmapping = num_frames/10
else:
  "ATTENTION: You need at least 100 frames to proceed, please increase your MD simulation time."

u2.trajectory[0] # set u2 to first frame

# print(rms.rmsd(u1.select_atoms('name CA').positions, u2.select_atoms('name CA').positions, superposition=False))

align.AlignTraj(u1, u2, select=mask_mda, in_memory=True).run()

nw = u1.select_atoms(mask_mda)

#Remove the Waters
with mda.Writer(nw_dcd, nw.n_atoms) as W:
  for ts in u1.trajectory[::int(Skip)]:
      W.write(nw, )
not_waters = u2.select_atoms(mask_mda)
not_waters.write(nw_pdb)
traj_dcd_check = os.path.exists(nw_dcd)
traj = nw_dcd
pdb_ref = nw_pdb
#For Backmapping
# with mda.Writer(backmap_dcd, nw.n_atoms) as W:
#   for ts in u1.trajectory[::int(stride_backmapping)]:
#   # for ts in u1.trajectory[0:100:int(1)]:
#       W.write(nw, )
#keep the waters
with mda.Writer(whole_dcd, u1.select_atoms("all").n_atoms) as W:
  for ts in u1.trajectory[::int(Skip)]:
      W.write(u1.select_atoms("all"))
whole = u2.select_atoms("all")
whole.write(whole_pdb)
traj_dcd_check = os.path.exists(whole_dcd)
  # traj = whole_dcd
  # pdb_ref = whole_pdb

traj_load_cg = pt.load(traj, pdb_ref)
print(traj_load_cg)

if traj_dcd_check == True:
  print("Trajectory concatenated successfully! :-)")
else:
  print("ERROR: Check your inputs! ")

# **Load, view and check the trajectory**
This will take a few minutes. 

In [ ]:

import warnings
warnings.filterwarnings('ignore')

#py3dmol functions
class Atom(dict):
  def __init__(self, line):
    self["type"] = line[0:6].strip()
    self["idx"] = line[6:11].strip()
    self["name"] = line[12:16].strip()
    self["resname"] = line[17:20].strip()
    self["resid"] = int(int(line[22:26]))
    self["x"] = float(line[30:38])
    self["y"] = float(line[38:46])
    self["z"] = float(line[46:54])
    self["sym"] = line[76:78].strip()

  def __str__(self):
    line = list(" " * 80)
    line[0:6] = self["type"].ljust(6)
    line[6:11] = self["idx"].ljust(5)
    line[12:16] = self["name"].ljust(4)
    line[17:20] = self["resname"].ljust(3)
    line[22:26] = str(self["resid"]).ljust(4)
    line[30:38] = str(self["x"]).rjust(8)
    line[38:46] = str(self["y"]).rjust(8)
    line[46:54] = str(self["z"]).rjust(8)
    line[76:78] = self["sym"].rjust(2)
    return "".join(line) + "\n"

class Molecule(list):
  def __init__(self, file):
    for line in file:
      if "ATOM" in line or "HETATM" in line:
        self.append(Atom(line))

    def __str__(self):
      outstr = ""
      for at in self:
        outstr += str(at)
      return outstr

if number_frames_analysis > 10:
  stride_animation = number_frames_analysis/10
else:
  stride_animation = 1

u = mda.Universe(pdb_ref, traj)

# Write out frames for animation
protein = u.select_atoms('all')
i = 0
for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]:
    if i > -1:
        with mda.Writer('' + str(i) + '.pdb', protein.n_atoms) as W:
            W.write(protein)
    i = i + 1
# Load frames as molecules
molecules = []
for i in range(int(len(u.trajectory)/int(stride_animation))):
    with open('' + str(i) + '.pdb') as ifile:
        molecules.append(Molecule(ifile))

models = ""
for i in range(len(molecules)):
  models += "MODEL " + str(i) + "\n"
  for j,mol in enumerate(molecules[i]):
    models += str(mol)
  models += "ENDMDL\n"
#view.addModelsAsFrames(models)

# Animation
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(models)
for i, at in enumerate(molecules[0]):
    default = {"sphere": {'color': 'spectrum'}}
    view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))

view.zoomTo()
view.animate({'loop': "forward"})
view.show()


# **Back-mapping**

Here we are going to use a general approach to transforming molecular models between different levels of resolution, based on machine learning methods. To perform this transformation we will use the [cg2all](https://www.biorxiv.org/content/10.1101/2023.05.22.541652v1) code.

## **Conversion of a CG simulation trajectory to an atomistic simulation trajectory using [__convert_cg2all__](https://github.com/huhlim/cg2all/tree/main)**

Back-map a coarse-grained model (trajectory) to a finer level of representation (atomistic).

This step will take a few minutes. it will be proportional to the number of frames you have in your final coarse-grained trajectory.


### **Parameters for conversion**

In [ ]:
Path = '/app/MD_martini/AKP6_WT'
restart = False
if restart: Jobname = 'AKP6_WT' 
Coarse_grained_model = "Martini3" # Martini, Martini3
batch_size = 1  # Batch size should be a divisor of the total number of frames.
device = "cuda" # cpu, cuda

In [ ]:
!source activate cg2all
#python
import cg2all.lib.libmodel
from cg2all.lib.libconfig import MODEL_HOME
for model_type in ["Martini", "Martini3",]:
    ckpt_fn = MODEL_HOME / f"{model_type}.ckpt"
    if not ckpt_fn.exists():
        cg2all.lib.libmodel.download_ckpt_file(model_type, ckpt_fn)

import os
import subprocess
import sys

workDir = Path
traj_input = os.path.join(workDir, str(Jobname) + "_cg_nw.dcd")
pdb_input = os.path.join(workDir, str(Jobname) +  "_nw.pdb")
pdb_ini_output = os.path.join(workDir, str(Jobname) +  "_frame0_backmapped.pdb")
traj_output = os.path.join(workDir, str(Jobname) +  "_backmapped.dcd")
pdb_output = os.path.join(workDir, str(Jobname) +  "_backmapped.pdb")
backmap_align_dcd = os.path.join(workDir, str(Jobname) +  "_backmapped_align.dcd")
backmap_align_pdb = os.path.join(workDir, str(Jobname) +  "_backmapped_align.pdb")

subprocess.run("convert_cg2all -p " + str(pdb_input) + " -o " + str(pdb_ini_output) + " --cg " + str(Coarse_grained_model), shell=True)
subprocess.run("convert_cg2all -p " + str(pdb_input) + " --dcd " + str(traj_input) + " -o " + str(traj_output) + " -opdb " + str(pdb_output) + " --cg " + str(Coarse_grained_model) + " --batch " + str(batch_size), shell=True)
# print(f"Converted {traj_input} in Martini to {traj_output}")


# traj_dcd_check = os.path.exists(traj_output)

# if traj_dcd_check == True:
#   print("Trajectory back-mapped successfully! :-)")
#   print("Number of frames:", num_frames)
# else:
#   print("ERROR: Check your inputs! ")

## **Align the backmapped trajectory**

### **Parameters for the alignment**

In [ ]:
Path = '/app/MD_martini/AKP6_WT'
restart = False
if restart: Jobname = 'AKP6_WT' 

In [ ]:
workDir = Path

pdb_ini_output = os.path.join(workDir, str(Jobname) +  "_frame0_backmapped.pdb")
traj_output = os.path.join(workDir, str(Jobname) +  "_backmapped.dcd")
pdb_output = os.path.join(workDir, str(Jobname) +  "_backmapped.pdb")
backmap_align_dcd = os.path.join(workDir, str(Jobname) +  "_backmapped_align.dcd")
backmap_align_pdb = os.path.join(workDir, str(Jobname) +  "_backmapped_align.pdb")


u1 = mda.Universe(pdb_ini_output, traj_output)
u2 = mda.Universe(pdb_ini_output, pdb_ini_output)

u2.trajectory[0] # set u2 to first frame

align.AlignTraj(u1, u2, select='name CA', in_memory=True).run()

sele = u1.select_atoms("all")

with mda.Writer(backmap_align_dcd, sele.n_atoms) as W:
    for ts in u1.trajectory[::]:
        W.write(sele, )
sele2 = u2.select_atoms("all")
sele2.write(backmap_align_pdb)
traj_dcd_check = os.path.exists(backmap_align_dcd)
traj_atomistic = backmap_align_dcd
pdb_ref_atomistic = backmap_align_pdb

# Calculate the length of the trajectory
u = mda.Universe(pdb_ref_atomistic,traj_atomistic)
num_frames = len(u.trajectory)

traj_load_atomistic = pt.load(traj_atomistic, pdb_ref_atomistic)

if traj_dcd_check == True:
    print("Trajectory aligned successfully! :-)")
    print("Number of frames:", num_frames)
else:
    print("ERROR: Check your inputs! ")

## **Load, view and check the back-mapped trajectory**
This will take a few minutes.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

#py3dmol functions
class Atom(dict):
    def __init__(self, line):
        self["type"] = line[0:6].strip()
        self["idx"] = line[6:11].strip()
        self["name"] = line[12:16].strip()
        self["resname"] = line[17:20].strip()
        self["resid"] = int(int(line[22:26]))
        self["x"] = float(line[30:38])
        self["y"] = float(line[38:46])
        self["z"] = float(line[46:54])
        self["sym"] = line[76:78].strip()

    def __str__(self):
        line = list(" " * 80)
        line[0:6] = self["type"].ljust(6)
        line[6:11] = self["idx"].ljust(5)
        line[12:16] = self["name"].ljust(4)
        line[17:20] = self["resname"].ljust(3)
        line[22:26] = str(self["resid"]).ljust(4)
        line[30:38] = str(self["x"]).rjust(8)
        line[38:46] = str(self["y"]).rjust(8)
        line[46:54] = str(self["z"]).rjust(8)
        line[76:78] = self["sym"].rjust(2)
        return "".join(line) + "\n"

class Molecule(list):
    def __init__(self, file):
        for line in file:
            if "ATOM" in line or "HETATM" in line:
                self.append(Atom(line))

        def __str__(self):
            outstr = ""
            for at in self:
                outstr += str(at)
            return outstr

backmap_align_dcd = os.path.join(workDir, str(Jobname) +  "_backmapped_align.dcd")
backmap_align_pdb = os.path.join(workDir, str(Jobname) +  "_backmapped_align.pdb")


number_frames_analysis = num_frames

if number_frames_analysis > 10:
    stride_animation = number_frames_analysis/10
else:
    stride_animation = 1

u = mda.Universe(backmap_align_pdb,backmap_align_dcd)

# Write out frames for animation
protein = u.select_atoms('all')
i = 0
for ts in u.trajectory[0:len(u.trajectory):int(stride_animation)]:
    if i > -1:
        with mda.Writer('' + str(i) + '.pdb', protein.n_atoms) as W:
            W.write(protein)
    i = i + 1
# Load frames as molecules
molecules = []
for i in range(int(len(u.trajectory)/int(stride_animation))):
    with open('' + str(i) + '.pdb') as ifile:
        molecules.append(Molecule(ifile))

models = ""
for i in range(len(molecules)):
    models += "MODEL " + str(i) + "\n"
    for j,mol in enumerate(molecules[i]):
        models += str(mol)
    models += "ENDMDL\n"
#view.addModelsAsFrames(models)

# Animation
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(models)
for i, at in enumerate(molecules[0]):
    default = {"cartoon": {'color': 'spectrum'}}
    view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))

view.zoomTo()
view.animate({'loop': "forward"})
view.show()

# **Analysis**

## **Compute protein RMSD**
**Provide output file names below:**

In [ ]:
Output_name = 'rmsd'

In [ ]:
nw_dcd = os.path.join(workDir, str(Jobname) + "_cg_nw.dcd")
nw_pdb = os.path.join(workDir, str(Jobname) +  "_nw.pdb")
traj_load_cg = pt.load(nw_dcd, nw_pdb)
traj_atomistic = os.path.join(workDir, str(Jobname) +  "_backmapped_align.dcd")
pdb_ref_atomistic = os.path.join(workDir, str(Jobname) +  "_backmapped_align.pdb")
traj_load_atomistic = pt.load(traj_atomistic, pdb_ref_atomistic)

ref_top = pt.load(pdb_ref_atomistic, pdb_ref_atomistic)
rmsd_cg = pt.rmsd(traj_load_cg, ref = 0)
df = pd.Series(rmsd_cg)
running_aver_cg = df.rolling(window =10).mean()
rmsd_atomistic = pt.rmsd(traj_load_atomistic, ref = 0, mask = "@CA,C,O,N,H")
df = pd.Series(rmsd_atomistic)
running_aver_atom = df.rolling(window =10).mean()


time = len(rmsd_cg)*int(Write_trajectory)/1000
time_array = np.arange(0,time,int(Write_trajectory)/1000)*int(stride_traj)

# Plotting:
fig, axs = plt.subplots(2, 2, figsize=(16, 5))
plt.subplot(1, 2, 1)
ax = plt.plot(time_array, rmsd_cg, alpha=0.2, color = 'blue', linewidth = 1.0)
ax = plt.plot(time_array, running_aver_cg, alpha=0.8, color = 'blue', linewidth = 1.0)
plt.xlim(0, simulation_ns)
plt.title("Coarse-grained")
plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSD [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.subplot(1, 2, 2)
ax = plt.plot(time_array, rmsd_atomistic, alpha=0.2, color = 'blue', linewidth = 1.0)
ax = plt.plot(time_array, running_aver_atom, alpha=0.8, color = 'blue', linewidth = 1.0)
plt.xlim(0, simulation_ns)
plt.title("Back-mapped")
plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSD [$\AA$]", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(rmsd_cg)
raw_data.to_csv(os.path.join(workDir, Output_name + "_cg.csv"))
raw_data=pd.DataFrame(rmsd_atomistic)
raw_data.to_csv(os.path.join(workDir, Output_name + "_atomistic.csv"))

## **Plot RMSD as a ditribution**

**Provide output file names below:**

In [ ]:
Output_name = 'rmsd_dist'

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 5))
plt.subplot(1, 2, 1)
ax = sb.kdeplot(rmsd_cg, color="blue", fill=True, alpha=0.2, linewidth=0.5)
plt.title("Coarse-grained")
plt.xlabel('RMSD [$\AA$]', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.subplot(1, 2, 2)
ax = sb.kdeplot(rmsd_atomistic, color="blue", fill=True, alpha=0.2, linewidth=0.5)
plt.title("Back-mapped")
plt.xlabel('RMSD [$\AA$]', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

## **Compute protein radius of gyration**

**Provide output file names below:**

In [ ]:
Output_name = 'radius_gyration'

In [ ]:
radgyr_cg = pt.radgyr(traj_load_cg)
df = pd.Series(radgyr_cg)
running_aver_cg = df.rolling(window =10).mean()
radgyr_atom = pt.radgyr(traj_load_atomistic)
df = pd.Series(radgyr_atom)
running_aver_atom = df.rolling(window =10).mean()

time = len(running_aver_cg)*int(Write_trajectory)/1000
time_array = np.arange(0,time,int(Write_trajectory)/1000)*int(stride_traj)

# Plotting:
fig, axs = plt.subplots(2, 2, figsize=(16, 5))
plt.subplot(1, 2, 1)
plt.plot(time_array, radgyr_cg, alpha=0.2, color = 'green', linewidth = 1.0)
plt.plot(time_array, running_aver_cg, alpha=0.8, color = 'green', linewidth = 1.0)
plt.xlim(0, simulation_ns)
plt.title("Coarse-grained")
plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Radius of gyration ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.subplot(1, 2, 2)
plt.plot(time_array, radgyr_atom, alpha=0.2, color = 'green', linewidth = 1.0)
plt.plot(time_array, running_aver_atom, alpha=0.8, color = 'green', linewidth = 1.0)
plt.xlim(0, simulation_ns)
plt.title("Back-mapped")
plt.xlabel("Time (ns)", fontsize = 14, fontweight = 'bold')
plt.ylabel("Radius of gyration ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')
raw_data=pd.DataFrame(radgyr_cg)
raw_data.to_csv(os.path.join(workDir, Output_name + "_cg.csv"))
raw_data=pd.DataFrame(radgyr_atom)
raw_data.to_csv(os.path.join(workDir, Output_name + "_atomistic.csv"))

## **Plot radius of gyration as a ditribution**

**Provide output file names below:**

In [ ]:
Output_name = 'radius_gyration_dist'

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 5))
plt.subplot(1, 2, 1)
ax = sb.kdeplot(radgyr_cg, color="green", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('Radius of gyration ($\AA$)', fontsize = 14, fontweight = 'bold')
plt.title("Coarse-grained")
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.subplot(1, 2, 2)
ax = sb.kdeplot(radgyr_atom, color="green", shade=True, alpha=0.2, linewidth=0.5)
plt.xlabel('Radius of gyration ($\AA$)', fontsize = 14, fontweight = 'bold')
plt.title("Back-mapped")
plt.xticks(fontsize = 12)
plt.yticks([])
plt.ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(False)

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

## **Compute RMSF of protein**

**Provide output file names below:**

In [ ]:
Output_name = 'rmsf'

In [ ]:
rmsf_cg = pt.rmsf(traj_load_cg, "byres")
# df = pd.Series(rmsf_cg[:,1])
# running_aver_cg = df.rolling(window =5).mean()
bfactor_cg = pt.bfactors(traj_load_cg, byres=False)
rmsf_atom = pt.rmsf(traj_load_atomistic,"byres")
bfactor_atom = pt.bfactors(traj_load_atomistic, byres=True)



# Plotting:
fig, axs = plt.subplots(2, 2, figsize=(16, 5))
plt.subplot(1, 2, 1)
plt.plot(rmsf_cg[:,1], alpha=1.0, color = 'red', linewidth = 1.0)
# plt.plot(running_aver_cg, alpha=1.0, color = 'red', linewidth = 1.0)
plt.title("Coarse-grained")
plt.xlabel("Residue", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSF ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.xlim(0, len(rmsf_cg[:-1]))

plt.subplot(1, 2, 2)
plt.plot(rmsf_atom[:,1], alpha=1.0, color = 'red', linewidth = 1.0)
plt.title("Back-mapped")
plt.xlabel("Residue", fontsize = 14, fontweight = 'bold')
plt.ylabel("RMSF ($\AA$)", fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.xlim(0, len(rmsf_atom[:-1]))

plt.yticks(fontsize = 12)
plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(rmsf_cg)
raw_data.to_csv(os.path.join(workDir, Output_name + "_cg.csv"))
raw_data=pd.DataFrame(rmsf_atom)
raw_data.to_csv(os.path.join(workDir, Output_name + "_atomistic.csv"))

## **2D RMSD**

**Provide output file names below:**

In [ ]:
Output_name = '2D_rmsd'

In [ ]:
last_frame = len(time_array)

stride_ticks_f = (last_frame)/5
ticks_frame = np.arange(0,(len(time_array) + float(stride_ticks_f)), float(stride_ticks_f))
a = ticks_frame.astype(float)
stride_ticks_t = (simulation_ns)/5
tick_time = np.arange(0,(float(simulation_ns) + float(stride_ticks_t)), float(stride_ticks_t))
b = tick_time.astype(float)

mat_cg = pt.pairwise_rmsd(traj_load_cg, frame_indices=range(int(number_frames_analysis)), metric="rms")
mat_atom = pt.pairwise_rmsd(traj_load_atomistic, mask = "@CA,C,O,N,H", frame_indices=range(int(number_frames_analysis)), metric="rms")

fig, axs = plt.subplots(2, 2, figsize=(16, 5))
plt.subplot(1, 2, 1)
ax = plt.imshow(mat_cg, cmap = 'PRGn', origin='lower', interpolation = 'bicubic')
plt.title("Coarse-grained")
plt.xlabel('Time (ns)', fontsize = 14, fontweight = 'bold')
plt.ylabel('Time (ns)', fontsize = 14, fontweight = 'bold')
plt.xticks(a, b.round(decimals=3), fontsize = 12)
plt.yticks(a, b.round(decimals=3), fontsize = 12)
# plt.clim(0,3)
cbar1 = plt.colorbar()
cbar1.set_label("RMSD ($\AA$)", fontsize = 14, fontweight = 'bold')


plt.subplot(1, 2, 2)
ax = plt.imshow(mat_atom, cmap = 'PRGn', origin='lower', interpolation = 'bicubic')
plt.title("Back-mapped")
plt.xlabel('Time (ns)', fontsize = 14, fontweight = 'bold')
plt.ylabel('Time (ns)', fontsize = 14, fontweight = 'bold')
plt.xticks(a, b.round(decimals=3), fontsize = 12)
plt.yticks(a, b.round(decimals=3), fontsize = 12)
# plt.clim(0,3)
cbar1 = plt.colorbar()
cbar1.set_label("RMSD ($\AA$)", fontsize = 14, fontweight = 'bold')

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(mat_cg)
raw_data.to_csv(os.path.join(workDir, Output_name + "_cg.csv"))
raw_data=pd.DataFrame(mat_atom)
raw_data.to_csv(os.path.join(workDir, Output_name + "_atomistic.csv"))

## **Calculate eigvenctors of Principle Component Analysis (PCA)**


**Provide output file names below:**

In [ ]:
Output_name = 'PCA'
Output_PC1 = 'PC1' 
Output_PC2 = 'PC2'

In [ ]:
data_cg = pt.pca(traj_load_cg, fit=True, ref=0, mask='', n_vecs=2)
data_atom = pt.pca(traj_load_atomistic, fit=True, ref=0, mask = "@CA,C,O,N,H", n_vecs=2)


last_frame = len(time_array)

stride_ticks_f = (last_frame)/5
ticks_frame = np.arange(0,(len(time_array) + float(stride_ticks_f)), float(stride_ticks_f))
a = ticks_frame.astype(float)
a2 = a.tolist()
stride_ticks_t = (simulation_ns)/5
tick_time = np.arange(0,(float(simulation_ns) + float(stride_ticks_t)), float(stride_ticks_t))
b = tick_time.astype(float)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # high resolution
fig, axs = plt.subplots(2, 2, figsize=(16, 5))
plt.subplot(1, 2, 1)
projection_data = data_cg[0]
plt.title("Coarse-grained")
PC1_cg = data_cg[0][0]
PC2_cg = data_cg[0][1]
a = plt.scatter(PC1_cg,PC2_cg, c=range(int(number_frames_analysis)), cmap='Greens', marker='o',s=8, alpha=1)
plt.clim(0, last_frame)
plt.xlabel('PC1', fontsize = 14, fontweight = 'bold')
plt.ylabel('PC2', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
cbar1 = plt.colorbar(a, orientation="vertical")
cbar1.set_label('Time(ns)', fontsize = 14, fontweight = 'bold')
cbar1.set_ticks(a2)
cbar1.set_ticklabels(b.round(decimals=3))

plt.subplot(1, 2, 2)
projection_data = data_atom[0]
plt.title("Back-mapped")
PC1_atom = data_atom[0][0]
PC2_atom = data_atom[0][1]
a = plt.scatter(PC1_atom,PC2_atom, c=range(int(number_frames_analysis)), cmap='Greens', marker='o',s=8, alpha=1)
plt.clim(0, last_frame)
plt.xlabel('PC1', fontsize = 14, fontweight = 'bold')
plt.ylabel('PC2', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
cbar1 = plt.colorbar(a, orientation="vertical")
cbar1.set_label('Time(ns)', fontsize = 14, fontweight = 'bold')
cbar1.set_ticks(a2)
cbar1.set_ticklabels(b.round(decimals=3))

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

pc1=pd.DataFrame(PC1_cg)
pc1.to_csv(os.path.join(workDir, Output_PC1 + "_cg.csv"))
pc2=pd.DataFrame(PC2_cg)
pc2.to_csv(os.path.join(workDir, Output_PC2 + "_cg.csv"))
pc1=pd.DataFrame(PC1_atom)
pc1.to_csv(os.path.join(workDir, Output_PC1 + "_atomistic.csv"))
pc2=pd.DataFrame(PC2_atom)
pc2.to_csv(os.path.join(workDir, Output_PC2 + "__atomistic.csv"))

## **Pearson's Cross Correlation (CC)**
**Provide output file names below:**


In [ ]:
Output_name = 'cross_correlation'

In [ ]:
cc_cg = matrix.correl(traj_load_cg)
cc_atom = matrix.correl(traj_load_atomistic, '@CA')

fig, axs = plt.subplots(2, 2, figsize=(16, 5))
plt.subplot(1, 2, 1)
ax = plt.imshow(cc_cg, cmap = 'PiYG_r', interpolation = 'bicubic', vmin = -1, vmax = 1, origin='lower')
plt.title("Coarse-grained")
plt.xlabel('Beads', fontsize = 14, fontweight = 'bold')
plt.ylabel('Beads', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
cbar1 = plt.colorbar()
cbar1.set_label('$CC_ij$', fontsize = 14, fontweight = 'bold')

plt.subplot(1, 2, 2)
ax = plt.imshow(cc_atom, cmap = 'PiYG_r', interpolation = 'bicubic', vmin = -1, vmax = 1, origin='lower')
plt.title("Back-mapped")
plt.xlabel('Residues', fontsize = 14, fontweight = 'bold')
plt.ylabel('Residues', fontsize = 14, fontweight = 'bold')
plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)
cbar1 = plt.colorbar()
cbar1.set_label('$CC_ij$', fontsize = 14, fontweight = 'bold')

plt.savefig(os.path.join(workDir, Output_name + ".png"), dpi=600, bbox_inches='tight')

raw_data=pd.DataFrame(cc_cg)
raw_data.to_csv(os.path.join(workDir, Output_name + "_cg.csv"))
raw_data=pd.DataFrame(cc_atom)
raw_data.to_csv(os.path.join(workDir, Output_name + "_atomistic.csv"))